In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('week_5_homework') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/27 22:56:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Question 1: 

**Install Spark and PySpark** 

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

What's the output?
- 3.3.2
- 2.1.4
- 1.2.3
- 5.4
</br></br>

In [3]:
spark.version

'3.3.2'

### Question 2: 

**HVFHW June 2021**

Read it with Spark using the same schema as we did in the lessons.</br> 
We will use this dataset for all the remaining questions.</br>
Repartition it to 12 partitions and save it to parquet.</br>
What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.</br>


- 2MB
- 24MB
- 100MB
- 250MB
</br></br>

In [3]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [5]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv("../data/raw/fhvhv/2021/06/fhvhv_tripdata_2021-06.csv.gz")

In [6]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [7]:
df.repartition(12, "pickup_datetime") \
    .write.parquet("../data/pq/fhvhv/2021/06/", mode='overwrite')

In [8]:
df.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
+--------------------+------------------

In [9]:
# df.write.parquet("../data/pq/fhvhv/2021/06/")

In [10]:
!ls -l ../data/pq/fhvhv/2021/06 | grep part

-rw-r--r-- 1 502 dialout 16683205 Feb 27 21:15 part-00000-73472ac9-654e-4909-8b68-1464d33861c2-c000.snappy.parquet
-rw-r--r-- 1 502 dialout 16817539 Feb 27 21:15 part-00001-73472ac9-654e-4909-8b68-1464d33861c2-c000.snappy.parquet
-rw-r--r-- 1 502 dialout 16808151 Feb 27 21:15 part-00002-73472ac9-654e-4909-8b68-1464d33861c2-c000.snappy.parquet
-rw-r--r-- 1 502 dialout 16778137 Feb 27 21:15 part-00003-73472ac9-654e-4909-8b68-1464d33861c2-c000.snappy.parquet
-rw-r--r-- 1 502 dialout 16786976 Feb 27 21:16 part-00004-73472ac9-654e-4909-8b68-1464d33861c2-c000.snappy.parquet
-rw-r--r-- 1 502 dialout 16744925 Feb 27 21:16 part-00005-73472ac9-654e-4909-8b68-1464d33861c2-c000.snappy.parquet
-rw-r--r-- 1 502 dialout 16825490 Feb 27 21:16 part-00006-73472ac9-654e-4909-8b68-1464d33861c2-c000.snappy.parquet
-rw-r--r-- 1 502 dialout 16764827 Feb 27 21:16 part-00007-73472ac9-654e-4909-8b68-1464d33861c2-c000.snappy.parquet
-rw-r--r-- 1 502 dialout 16725136 Feb 27 21:16 part-00008-73472ac9-654e-4909-8b6

In [11]:
!ls -l ../data/pq/fhvhv/2021/06 | grep part |awk '{sum +=$5; n++} END {printf "%.0fMB\n", sum/n/1024/1024}'

16MB


### Question 3: 

**Count records**  

How many taxi trips were there on June 15?</br></br>
Consider only trips that started on June 15.</br>

- 308,164
- 12,856
- 452,470
- 50,982
</br></br>

In [3]:
df_pq = spark.read.parquet('../data/pq/fhvhv/2021/06/')

In [6]:
df_pq \
    .withColumn('pickup_date', F.to_date(df_pq.pickup_datetime)) \
    .filter("pickup_date = '2021-06-15'") \
    .count()

452470

### Question 4: 

**Longest trip for each day**  

Now calculate the duration for each trip.</br>
How long was the longest trip in Hours?</br>

- 66.87 Hours
- 243.44 Hours
- 7.68 Hours
- 3.32 Hours
</br></br>

In [5]:
df_pq.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:49:01|2021-06-01 01:03:50|         182|         259|      N|                  null|
|              B02510|2021-06-01 00:07:36|2021-06-01 00:21:13|         188|          72|      N|                  null|
|              B02764|2021-06-01 00:42:12|2021-06-01 00:50:28|         186|          48|      N|                B02764|
|              B02510|2021-06-01 00:11:38|2021-06-01 00:18:53|          47|         159|      N|                  null|
+--------------------+------------------

In [11]:
df_pq \
    .withColumn('duration', (df_pq.dropoff_datetime.cast('long') - df_pq.pickup_datetime.cast('long'))/3600) \
    .withColumn('pickup_date', F.to_date(df_pq.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+------------------+
|pickup_date|     max(duration)|
+-----------+------------------+
| 2021-06-25|  66.8788888888889|
| 2021-06-22|25.549722222222222|
| 2021-06-27|19.980833333333333|
| 2021-06-26|18.197222222222223|
| 2021-06-23|16.466944444444444|
+-----------+------------------+



In [8]:
df_pq.createOrReplaceTempView('fhvhv_2021_06')

In [18]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX(round((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600,2)) AS duration
FROM 
    fhvhv_2021_06
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+--------+
|pickup_date|duration|
+-----------+--------+
| 2021-06-25|   66.88|
| 2021-06-22|   25.55|
| 2021-06-27|   19.98|
| 2021-06-26|    18.2|
| 2021-06-23|   16.47|
| 2021-06-24|   13.91|
| 2021-06-04|   11.67|
| 2021-06-20|   10.98|
| 2021-06-01|   10.27|
| 2021-06-28|    9.97|
+-----------+--------+



> The answer is : 66.88 hours

### Question 5: 

**User Interface**

 Spark’s User Interface which shows application's dashboard runs on which local port?</br>

- 80
- 443
- 4040
- 8080
</br></br>

> The answer is : 8080

### Question 6: 

**Most frequent pickup location zone**

Load the zone lookup data into a temp view in Spark</br>
[Zone Data](https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv)</br>

Using the zone lookup data and the fhvhv June 2021 data, what is the name of the most frequent pickup location zone?</br>

- East Chelsea
- Astoria
- Union Sq
- Crown Heights North
</br></br>

In [4]:
df_zones = spark.read.option("header","true").csv('../data/taxi_zone_lookup.csv')

In [5]:
df_zones.show(4)

+----------+---------+--------------------+------------+
|LocationID|  Borough|                Zone|service_zone|
+----------+---------+--------------------+------------+
|         1|      EWR|      Newark Airport|         EWR|
|         2|   Queens|         Jamaica Bay|   Boro Zone|
|         3|    Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|Manhattan|       Alphabet City| Yellow Zone|
+----------+---------+--------------------+------------+
only showing top 4 rows



In [6]:
df_zones.createOrReplaceTempView('zones')

In [9]:
spark.sql("""
SELECT
    CONCAT(pul.Zone, ' / ', dol.Zone) AS pu_do_pair,
    COUNT(1) as frequency
FROM 
    fhvhv_2021_06 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+--------------------+--------+
|          pu_do_pair|count(1)|
+--------------------+--------+
|East New York / E...|   47926|
|    JFK Airport / NA|   31321|
| Canarsie / Canarsie|   28230|
|Crown Heights Nor...|   25216|
|Borough Park / Bo...|   24778|
+--------------------+--------+



In [12]:
spark.sql("""
SELECT
    pul.Zone as pickup_zone,
    COUNT(1) as frequency
FROM 
    fhvhv_2021_06 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+-------------------+---------+
|        pickup_zone|frequency|
+-------------------+---------+
|Crown Heights North|   231279|
|       East Village|   221244|
|        JFK Airport|   188867|
|     Bushwick South|   187929|
|      East New York|   186780|
+-------------------+---------+



> The answer is : Crown Heights North